# Literary text editions in the collection of the Antwerp University Library 

## Aim of the collection analysis

The aim of this document is to show how SQlite and Python can help with a collection analysis. We will focus on the literary text editions which are relevant for students and researchers of the Faculty of Arts, especially of the Literary Department. These items are after all the primary sources for their work.

The collection of literary text editions has been mainly built up by the Department of Literature and to a lesser extent by the Department of Linguistics. There were obviously also donations and other departments also bought, albeit to a very limited extent, some literary publications.

The catalog records of text editions contains a keyword to distinguish them from other publications. We will limit our queries to the following keywords: 'Tekstuitgave' (text editions), 'Theatertekst' (theater text), 'Poëzie' (poetry), 'Verhalend proza' (narrative prose), and 'Bloemlezing' (anthology).

Note: The cataloguing rules related to the keywords for text editions were changed in 2008. Until that year there was no distinction between theater text, poetry, etc. All text editions got the same general keyword: 'Text editions'. This means that a work of poetry before the change got the keyword 'text edition' and not the keyword 'poetry'. In short, it is not possible for example to focus the searches only to poetry, because in that case you will miss a lot of works of poetry in your result set.
  
## Dataset

This collection analysis uses bibliographic descriptions (catalog records) from the Antwerp University Library catalog, managed by the library system Brocade (dataset on 5/9/2022), and not the number of items (physical copies of a book). In other words, each description has one or more items attached to it. Keep in mind that a work (e.g. Het verdriet van België of Hugo Claus) may have several bibliographic descriptions.

The dataset is exported as a SQLite database and queried with the help of Python libraries (sqlite3, numpy, and pandas).

## 1 : Authors and translators 

- Topic 1: Which authors of text editions appear most frequently in the library collection (reading rooms and stack rooms)?
- Topic 2: The same question for the translators.
- Topic 3: Focus on the translations of Hugo Claus work: languages and titles (sample). 
- Topic 4: The translations of Williams Shakespeare work: focus on the Dutch translations.


Note: The cataloguing rules for authors changed in June 2013. The cataloguers now must record all authors/editors mentioned on the title page, whereas in the past they had to limit themselves to the first three authors/editors. However, this is not of a significant relevance because text editions with more than three authors are quite rare.




In [9]:
# Query for Topic 1

import sqlite3
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

PATH_TO_DB = '/Users/lmaggior/OneDrive - Universiteit Antwerpen/programmeren/sqlite/UA_cst.sqlite'

# PATH_TO_DB = '/Users/LCD150030/OneDrive - Universiteit Antwerpen/programmeren/sqlite/UA_cst.sqlite'

conn = sqlite3.connect(PATH_TO_DB)
cursor = conn.cursor()

'''
Function to generate pandas DataFrame's from the result set of the sqlite query
'''
def execute_cursor(query, columns_input):
    cursor.execute(query)
    data = {row for row in cursor.fetchall()}
    data_frame = pd.DataFrame(data, columns = columns_input)
    return data_frame

'''
Function to counts records and to stylize the result table
'''
def styling_frame_count (data_frame, column, head):
    counts = data_frame[column].value_counts()
    counts_head = counts.head(head)
    styling = counts_head.to_frame('Number of records').style.set_table_styles([dict(selector='th', props=[('text-align', 'left')])])
    alignment = styling.set_properties(**{'text-align': 'left'})
    display (alignment)
    
'''
Function to stylize the result table when no records have to be counted
'''  
    
def styling_frame (data_frame, head):
    if head == 0:
        styling = data_frame.style.set_table_styles([dict(selector='th', props=[('text-align', 'left')])])
        alignment = styling.set_properties(**{'text-align': 'left'})
        display (alignment)
    else:
        data_frame_head = data_frame.head(head)
        styling = data_frame_head.style.set_table_styles([dict(selector='th', props=[('text-align', 'left')])])
        alignment = styling.set_properties(**{'text-align': 'left'})
        display (alignment)
        
        
'''
Function to print the heading of the result table
'''        

def print_heading (text):
    print("\n \n \033[1m" + f'{text} \n \033[0m ')
    
    
query1 = '''

SELECT 
    token.cloi,
    title_ti as Title,
    author_fn || ', ' || author_vn AS Author 
FROM token 
JOIN 
    author ON token.cloi = author.cloi
JOIN 
    holding ON author.cloi = holding.cloi
JOIN 
    title ON holding.cloi = title.cloi
JOIN 
    subject ON subject.cloi = title.cloi
WHERE 
subject_ty LIKE 'PT' 
AND author_fu LIKE 'aut'
AND author_fn NOT LIKE 'e.a.'
AND holding.lib LIKE 'UA-CST'
AND title_ty LIKE 'h'
AND subject_vw LIKE 'Tekstuitgave'

UNION

SELECT 
    token.cloi,
    title_ti as Title,
    author_fn || ', ' || author_vn AS Author 
FROM token 
JOIN 
    author ON token.cloi = author.cloi
JOIN 
    holding ON author.cloi = holding.cloi
JOIN 
    title ON holding.cloi = title.cloi
JOIN 
    subject ON subject.cloi = title.cloi
WHERE 
subject_ty LIKE 'PT' 
AND author_fu LIKE 'aut'
AND author_fn NOT LIKE 'e.a.'
AND holding.lib like 'UA-CST'
AND title_ty LIKE 'h'
AND subject_vw LIKE 'Theatertekst'

UNION

SELECT 
    token.cloi,
    title_ti as Title,
    author_fn || ', ' || author_vn AS Author 
FROM token 
JOIN 
    author ON token.cloi = author.cloi
JOIN 
    holding ON author.cloi = holding.cloi
JOIN 
    title ON holding.cloi = title.cloi
JOIN 
    subject ON subject.cloi = title.cloi
WHERE 
subject_ty LIKE 'PT' 
AND author_fu LIKE 'aut'
AND author_fn NOT LIKE 'e.a.'
AND holding.lib LIKE 'UA-CST'
AND title_ty LIKE 'h'
AND subject_vw LIKE 'Poëzie'

UNION

SELECT 
    token.cloi,
    title_ti as Title,
    author_fn || ', ' || author_vn AS Author 
FROM token 
JOIN 
    author ON token.cloi = author.cloi
JOIN 
    holding ON author.cloi = holding.cloi
JOIN 
    title ON holding.cloi = title.cloi
JOIN 
    subject ON subject.cloi = title.cloi
WHERE 
subject_ty LIKE 'PT' 
AND author_fu LIKE 'aut'
AND author_fn NOT LIKE 'e.a.'
AND holding.lib LIKE 'UA-CST'
AND title_ty LIKE 'h'
AND subject_vw LIKE 'Verhalend proza'

UNION

SELECT 
    token.cloi,
    title_ti as Title,
    author_fn || ', ' || author_vn AS Author 
FROM token 
JOIN 
    author  ON token.cloi = author.cloi
JOIN 
    holding ON author.cloi = holding.cloi
JOIN 
    title ON holding.cloi = title.cloi
JOIN 
    subject ON subject.cloi = title.cloi
WHERE 
subject_ty LIKE 'PT' 
AND author_fu LIKE 'aut'
AND author_fn NOT LIKE 'e.a.'
AND holding.lib LIKE 'UA-CST'
AND title_ty LIKE 'h'
AND subject_vw LIKE 'Bloemlezing'

'''
print_heading('Authors of text editions in the Library collection (top 20)')
styling_frame_count(execute_cursor(query1, ['cloi', 'Title', 'Author']), 'Author', 20)

# Query for Topic 2

query2 = '''
SELECT 
    token.cloi,
    title_ti as Title,
    author_fn || ', ' || author_vn as Author 
FROM token 
JOIN 
    author  on token.cloi = author.cloi
JOIN 
    holding on author.cloi = holding.cloi
JOIN 
    title on holding.cloi = title.cloi
JOIN 
    subject on subject.cloi = title.cloi 
WHERE 
subject_ty like 'PT' 
AND author_fu like 'trl'
AND author_fn not like 'e.a.'
AND holding.lib like 'UA-CST'
AND title_ty like 'h'
AND subject_vw like 'Tekstuitgave'

UNION

SELECT 
    token.cloi,
    title_ti as Title,
    author_fn || ', ' || author_vn as Author 
FROM token 
JOIN 
    author  on token.cloi = author.cloi
JOIN 
    holding on author.cloi = holding.cloi
JOIN 
    title on holding.cloi = title.cloi
JOIN 
    subject on subject.cloi = title.cloi
WHERE 
subject_ty like 'PT' 
AND author_fu like 'trl'
AND author_fn not like 'e.a.'
AND holding.lib like 'UA-CST'
AND title_ty like 'h'
AND subject_vw like 'Theatertekst'

UNION

SELECT 
    token.cloi,
    title_ti as Title,
    author_fn || ', ' || author_vn as Author 
FROM token 
JOIN 
    author  on token.cloi = author.cloi
JOIN 
    holding on author.cloi = holding.cloi
JOIN 
    title on holding.cloi = title.cloi
JOIN 
    subject on subject.cloi = title.cloi
WHERE 
subject_ty like 'PT' 
AND author_fu like 'trl'
AND author_fn not like 'e.a.'
AND holding.lib like 'UA-CST'
AND title_ty like 'h'
AND subject_vw like 'Poëzie'

UNION

SELECT 
    token.cloi,
    title_ti as Title,
    author_fn || ', ' || author_vn as Author 
FROM token 
JOIN 
    author  on token.cloi = author.cloi
JOIN 
    holding on author.cloi = holding.cloi
JOIN 
    title on holding.cloi = title.cloi
JOIN 
    subject on subject.cloi = title.cloi
WHERE 
subject_ty like 'PT' 
AND author_fu like 'trl'
AND author_fn not like 'e.a.'
AND holding.lib like 'UA-CST'
AND title_ty like 'h'
AND subject_vw like 'Verhalend proza'

UNION

SELECT 
    token.cloi,
    title_ti as Title,
    author_fn || ', ' || author_vn as Author 
FROM token 
JOIN 
    author  on token.cloi = author.cloi
JOIN 
    holding on author.cloi = holding.cloi
JOIN 
    title on holding.cloi = title.cloi
JOIN 
    subject on subject.cloi = title.cloi
WHERE 
subject_ty like 'PT' 
AND author_fu like 'trl'
AND author_fn not like 'e.a.'
AND holding.lib like 'UA-CST'
AND title_ty like 'h'
AND subject_vw like 'Beeldverhaal'

UNION

SELECT 
    token.cloi,
    title_ti as Title,
    author_fn || ', ' || author_vn as Author 
FROM token 
JOIN 
    author  on token.cloi = author.cloi
JOIN 
    holding on author.cloi = holding.cloi
JOIN 
    title on holding.cloi = title.cloi
JOIN 
    subject on subject.cloi = title.cloi
WHERE 
subject_ty like 'PT' 
AND author_fu like 'trl'
AND author_fn not like 'e.a.'
AND holding.lib like 'UA-CST'
AND title_ty like 'h'
AND subject_vw like 'Bloemlezing'
'''
print_heading('Translators of text editions in the Library collection (top 20)')
styling_frame_count(execute_cursor(query2, ['cloi', 'Title', 'Author']), 'Author', 20)

# Query for Topic 3

query3 = '''
SELECT 
token.cloi,
title_ti as Title,
author.author_fn || ', ' || author.author_vn as Author,
author2.author_fn || ', ' || author2.author_vn as Translator,
language_lg as Language
FROM token 
JOIN 
author ON token.cloi = author.cloi
JOIN 
holding ON author.cloi = holding.cloi
JOIN 
title ON holding.cloi = title.cloi
JOIN 
subject ON subject.cloi = title.cloi
JOIN 
language ON subject.cloi = language.cloi
JOIN 
holding2 on holding2.cloi = language.cloi
JOIN 
author2 on holding2.cloi = author2.cloi
WHERE 
subject_ty LIKE 'PT'
AND author.author_fu LIKE 'aut'
AND Author LIKE 'Claus, Hugo'
AND author2.author_fu LIKE 'trl'
AND ((holding.lib LIKE 'UA-CST' AND holding.holding_pk LIKE 'T&L%') OR (holding2.lib LIKE 'UA-CST' AND holding2.holding_pk LIKE 'MAG%'))
AND title_ty LIKE 'h'
AND Language  NOT LIKE 'dut'
GROUP BY token.cloi
'''
print_heading('Claus works: languages of the translations')
styling_frame_count(execute_cursor(query3,  ['cloi', 'Title', 'Author', 'Translator', 'Language']), 'Language', 20)
print_heading('Claus works: titles and translators (sample of 20 titles)')
styling_frame(execute_cursor(query3, ['cloi', 'Title', 'Author', 'Translator', 'Language']), 20)

# Query for Topic 4

query4 = '''
SELECT 
title_ti as Title,
author.author_fn || ', ' || author.author_vn as Author,
author2.author_fn || ', ' || author2.author_vn as Translator 
FROM token 
JOIN 
author ON token.cloi = author.cloi
JOIN 
holding ON author.cloi = holding.cloi
JOIN 
title ON holding.cloi = title.cloi
JOIN 
subject ON subject.cloi = title.cloi
JOIN 
language ON subject.cloi = language.cloi
JOIN 
holding2 on holding2.cloi = language.cloi
JOIN 
author2 on holding2.cloi = author2.cloi
WHERE 
subject_ty LIKE 'PT'
AND author.author_fu LIKE 'aut'
AND Author LIKE 'Shakespeare, William'
AND author2.author_fu LIKE 'trl'
AND ((holding.lib LIKE 'UA-CST' AND holding.holding_pk LIKE 'T&L%') OR (holding2.lib LIKE 'UA-CST' AND holding2.holding_pk LIKE 'MAG%'))
AND title_ty LIKE 'h'
AND language_lg LIKE 'dut'
GROUP BY token.cloi
'''
print_heading(' Translations of Shakespeare works in Dutch')
styling_frame(execute_cursor(query4, ['Title', 'Author', 'Translator']), 0)




 
 Authors of text editions in the Library collection (top 20) 
  


,Number of records
"Claus, Hugo",657
"Grimm,",532
"Shakespeare, William",485
"Cicero, Marcus Tullius",432
"Fontaine, de La, Jean",414
"Vondel, van den, Joost",355
"Gezelle, Guido",324
"Boon, Louis Paul",305
"Walschap, Gerard",289
"Conscience, Hendrik",272



 
 Translators of text editions in the Library collection (top 20) 
  


,Number of records
"Guerne, Armel",33
"Burgersdijk, Leendert A.J.",31
"Courteaux, Willy",30
"Vondel, van den, Joost",30
"Claus, Hugo",28
"Durand, Pierre",24
"Chouraqui, André",23
"Castier, Jules",22
"Pressler, Mirjam",22
"Boutens, Pieter Cornelis",22



 
 Claus works: languages of the translations 
  


,Number of records
fre,36
ger,28
eng,17
swe,11
nor,7
spa,7
afr,5
pol,5
cze,5
ita,4



 
 Claus works: titles and translators (sample of 20 titles) 
  


,cloi,Title,Author,Translator,Language
0,c:lvd:558822,Jours de canicule,"Claus, Hugo","Buysse, Maddy",fre
1,c:lvd:558757,"Andréa, ou : La fiancée du matin : pièce en 4 actes","Claus, Hugo","Buysse, Maddy",fre
2,c:lvd:6863130,Fristelsen,"Claus, Hugo","Rasmussen, Egil",nor
3,c:lvd:3200357,Apropo de Dédé,"Claus, Hugo","Boureanu, Radu",rum
4,c:lvd:6864163,Die dans van die reiger,"Claus, Hugo","Stockenström, Wilma",afr
5,c:lvd:846114,Svärdfisken,"Claus, Hugo","Holmer, Per",swe
6,c:lvd:3199673,Skam: roman,"Claus, Hugo","Holmer, Per",swe
7,c:lvd:705402,Thyeste : tragédie d'après Sénèque,"Claus, Hugo","Buysse, Maddy",fre
8,c:lvd:621145,"Four Flemish poets : Hugo Claus, Gust Gils, Paul Snoek, Hughes C. Pernath","Claus, Hugo","Brown, Paul",eng
9,c:lvd:3252321,L'étonnement: roman,"Claus, Hugo","Buysse, Maddy",fre



 
  Translations of Shakespeare works in Dutch 
  


,Title,Author,Translator
0,Koning Lear,"Shakespeare, William","Albers, Frank"
1,Hamlet,"Shakespeare, William","Burgersdijk, Leendert A.J."
2,Coriolanus,"Shakespeare, William","Voeten, Bert"
3,King Lear,"Shakespeare, William","Willems, Jibbe"
4,Midzomernachtdroom,"Shakespeare, William","Courteaux, Willy"
5,Hamlet,"Shakespeare, William","Courteaux, Willy"
6,Maat voor maat,"Shakespeare, William","Burgersdijk, Leendert A.J."
7,Verzameld werk,"Shakespeare, William","Courteaux, Willy"
8,De storm,"Shakespeare, William","Albers, Frank"
9,De koopman van Venetië: drama in 5 bedrijven,"Shakespeare, William","Koster, Edward B."


### Commentary on section 1

#### Commentary on topic 1

We find in the top 20 numerous classical Roman and Greek authors such as Cicero, Plato, Aristotle, etc. Not surprising is the good score by Hugo Claus, Louis Paul Boon, Gerard Walschap, and Guido Gezelle. The university has indeed a Hugo Claus center, a Louis Paul Boon center, a Gerard Walschap Society and a Guido Gezelle center. We also find three French-speaking authors in the top 20: Jean de la Fontaine, Georges Simenon and Honoré de Balzac. 
Jean de la Fontaine's high ranking is due to Wim Gielen's collection of fable books that the library integrated in its holding a few years ago (https://anet.be/record/opaccoloiuantwerpen/co:ua:82).

#### Commentary on topic 2

At the top of the list we find Armel Guerne, translator of Grimm works in French, then Leendert Burgesdijk, the well-known translator of the entire work of the English playwright and poet William Shakespeare and after him we find Willy Corteaux, also a translator of Shakespeare (plays) but also and especially of the work of Euripides. 
Not surprisingly in the top 20 are the names of modern authors such as Hugo Claus and Gerrit Komrij (both also known as translators) and the presence of Joost van den Vondel who translated the works of Greek and Roman authors into Dutch.


#### Commentary on topic 3

The works of Claus are mostly translated as expected in French, German, and English, but in the library collection we also find many translations in less common languages as Turkish, Bulgarian, and Russian.


#### Commentary on topic 4

As established in the analysis of the translators, we find the names of Willy Courteaux, Leendert Burgersdijk and Gerrit Komrij in the list of shakespearean translations.



## 2 : Language of the content

- Topic 5: The aim is to map the most common languages in the collection of text editions. This query concerns the field 'content of the language' of the catalog records, not the field 'language of the title'. For example: the result set of a query for text editions in Dutch contains both the publications of Dutch and Flemish authors and the publications translated in Dutch.


In [3]:
# Query for Topic 5

query = '''
SELECT 
token.cloi,
language_lg as Language
FROM token 
JOIN 
author ON token.cloi = author.cloi
JOIN 
holding ON author.cloi = holding.cloi
JOIN 
title ON holding.cloi = title.cloi
JOIN 
subject ON subject.cloi = title.cloi
JOIN 
language ON subject.cloi = language.cloi
WHERE 
subject_ty LIKE 'PT'
AND subject_vw LIKE 'Tekstuitgave' 
AND holding.lib LIKE 'UA-CST'
AND title_ty LIKE 'h'

UNION

SELECT 
token.cloi,
language_lg as Language
FROM token 
JOIN 
author ON token.cloi = author.cloi
JOIN 
holding ON author.cloi = holding.cloi
JOIN 
title ON holding.cloi = title.cloi
JOIN 
subject ON subject.cloi = title.cloi
JOIN 
language ON subject.cloi = language.cloi
WHERE 
subject_ty LIKE 'PT'
AND subject_vw LIKE 'Theatertekst' 
AND holding.lib LIKE 'UA-CST'
AND title_ty LIKE 'h'

UNION

SELECT 
token.cloi,
language_lg as Language
FROM token 
JOIN 
author ON token.cloi = author.cloi
JOIN 
holding ON author.cloi = holding.cloi
JOIN 
title ON holding.cloi = title.cloi
JOIN 
subject ON subject.cloi = title.cloi
JOIN 
language ON subject.cloi = language.cloi
WHERE 
subject_ty LIKE 'PT'
AND subject_vw LIKE 'Poëzie' 
AND holding.lib LIKE 'UA-CST'
AND title_ty LIKE 'h'

UNION

SELECT 
token.cloi,
language_lg as Language
FROM token 
JOIN 
author ON token.cloi = author.cloi
JOIN 
holding ON author.cloi = holding.cloi
JOIN 
title ON holding.cloi = title.cloi
JOIN 
subject ON subject.cloi = title.cloi
JOIN 
language ON subject.cloi = language.cloi
WHERE 
subject_ty LIKE 'PT'
AND subject_vw LIKE 'Verhalend proza' 
AND holding.lib LIKE 'UA-CST'
AND title_ty LIKE 'h'

UNION

SELECT 
token.cloi,
language_lg as Language
FROM token 
JOIN 
author ON token.cloi = author.cloi
JOIN 
holding ON author.cloi = holding.cloi
JOIN 
title ON holding.cloi = title.cloi
JOIN 
subject ON subject.cloi = title.cloi
JOIN 
language ON subject.cloi = language.cloi
WHERE 
subject_ty LIKE 'PT'
AND subject_vw LIKE 'Beeldverhaal' 
AND holding.lib LIKE 'UA-CST'
AND title_ty LIKE 'h'

UNION

SELECT 
token.cloi,
language_lg as Language
FROM token 
JOIN 
author ON token.cloi = author.cloi
JOIN 
holding ON author.cloi = holding.cloi
JOIN 
title ON holding.cloi = title.cloi
JOIN 
subject ON subject.cloi = title.cloi
JOIN 
language ON subject.cloi = language.cloi
WHERE 
subject_ty LIKE 'PT'
AND subject_vw LIKE 'Bloemlezing' 
AND holding.lib LIKE 'UA-CST'
AND title_ty LIKE 'h'

'''
print_heading('Text editions: most common languages (top 20)')
styling_frame_count(execute_cursor(query, ['cloi', 'Language']), 'Language', 20)


 
 Text editions: most common languages (top 20) 
  


,Number of records
dut,34044
fre,25147
eng,15433
ger,13320
spa,9252
lat,3494
ita,2139
por,805
rum,526
grc,386


### Commentary on section 2

The library's collection is mainly focused on Dutch and French. English and German score significantly less but more than the remaining European and not European languages. 




## 3 : Autobiographies, correspondance, and diaries

- Topic 6: Who are the authors with the most egodocuments in the library catalog?
- Topic 7: Sample of 20 titles 


In [4]:
# Query for Topic 6 and 7

query = '''
SELECT DISTINCT
 token.cloi,
 title_ti as Title,
 author_fn || ', ' || author_vn as Author
FROM token
JOIN holding 
ON token.cloi = holding.cloi
JOIN subject 
ON holding.cloi = subject.cloi
JOIN author 
ON author.cloi = subject.cloi
JOIN title 
ON author.cloi = title.cloi
JOIN subject2
ON title.cloi = subject2.cloi  
WHERE 
 author_fu like 'aut'
AND subject.subject_ac like 'a::920%'
AND Author == subject.subject_vw
AND  title_ty like 'h'
AND subject2.subject_ac like 'a::pt.4%'
'''
print_heading('Authors with the most egodocuments in the library catalog (top 20)')
styling_frame_count(execute_cursor(query, ['cloi','Title','Author']), 'Author', 20)

print_heading('Sample of 20 titles')
styling_frame(execute_cursor(query, ['cloi','Title','Author']), 20)


 
 Authors with the most egodocuments in the library catalog (top 20) 
  


,Number of records
"Gide, André",41
"Kafka, Franz",38
"Yourcenar, Marguerite",30
"Kierkegaard, Søren",27
"Goethe, von, Johann Wolfgang",23
"Frank, Anne",21
"Zweig, Stefan",19
"Joyce, James",19
"Canetti, Elias",18
"Hofmannsthal, von, Hugo",17



 
 Sample of 20 titles 
  


,cloi,Title,Author
0,c:lvd:266897,"Écrits de prison, précédés de: La vie de sir Thomas More","More, Thomas"
1,c:lvd:3190389,Jetzt: Max Frisch,"Frisch, Max"
2,c:lvd:7098071,The basic writings of Bertrand Russell,"Russell, Bertrand"
3,c:lvd:55881,An apology for the life of Colley Cibber,"Cibber, Colley"
4,c:lvd:986547,Michael Kohlhaas (aus einer alten Chronik): mit Materialien,"Kleist, von, Heinrich"
5,c:lvd:131,De briefwisseling tussen Frederik van Eeden en Lodewijk van Deyssel,"Deyssel, van, Lodewijk"
6,c:lvd:274996,Waarde Van Lennep: brieven van de Schoolmeester,"Schoolmeester, den"
7,c:lvd:231870,Oeuvres complètes,"Courier de Méré, Paul-Louis"
8,c:lvd:1134686,Ava : mijn leven,"Gardner, Ava"
9,c:lvd:631014,Essais et mémoires,"Yourcenar, Marguerite"


### Commentary on section 3

#### Commentary on topic 6

The text editions of correspondence represent the largest part of the egodocuments. As a result, all authors whose correspondence has been published score very well in the ranking. Unfortunately, it is not possible to distinguish autobiographies from correspondence and diaries, because these editions are treated in the same way by the cataloguing rules.

#### Commentary on topic 7

On the basis of a sample it is difficult to draw conclusions. But the sample shows how varied the collection is. 


## 4 : Online available content

- topic 8: The aim of this query is to determine which providers supply the largest online content in the library catalog. With online content we mean full text access to the text editions. Abstracts or summaries are thus obviously not considered as full text. 


In [12]:
# Query for Topic 8

query = '''
SELECT DISTINCT
token.cloi,
title_ti as Title,
subject_vw as Subject,
fulltext_access as Access,
fulltext_loc as Provider,
fulltext_zurl as URL
FROM token
JOIN 
fulltext ON token.cloi = fulltext.cloi
JOIN 
title ON fulltext.cloi = title.cloi
JOIN 
holding ON title.cloi = holding.cloi
JOIN 
subject ON subject.cloi = holding.cloi
JOIN 
impressum ON subject.cloi = impressum.cloi
WHERE
fulltext_ty LIKE 'full'
AND subject_ty LIKE 'PT' 
AND Subject LIKE 'Theatertekst'
AND title_ty LIKE 'h'
AND (Access LIKE 'ua' OR Access LIKE 'anet' )
group by URL

UNION

SELECT DISTINCT
token.cloi,
title_ti as Title,
subject_vw as Subject,
fulltext_access as Access,
fulltext_loc as Provider,
fulltext_zurl as URL
FROM token
JOIN 
fulltext ON token.cloi = fulltext.cloi
JOIN 
title ON fulltext.cloi = title.cloi
JOIN 
holding ON title.cloi = holding.cloi
JOIN 
subject ON subject.cloi = holding.cloi
JOIN 
impressum ON subject.cloi = impressum.cloi
WHERE
fulltext_ty LIKE 'full'
AND subject_ty LIKE 'PT' 
AND Subject LIKE 'Tekstuitgave'
AND title_ty LIKE 'h'
AND (Access LIKE 'ua' OR Access LIKE 'anet' )
group by URL

UNION

SELECT DISTINCT
token.cloi,
title_ti as Title,
subject_vw as Subject,
fulltext_access as Access,
fulltext_loc as Provider,
fulltext_zurl as URL
FROM token
JOIN 
fulltext ON token.cloi = fulltext.cloi
JOIN 
title ON fulltext.cloi = title.cloi
JOIN 
holding ON title.cloi = holding.cloi
JOIN 
subject ON subject.cloi = holding.cloi
JOIN 
impressum ON subject.cloi = impressum.cloi
WHERE
fulltext_ty LIKE 'full'
AND subject_ty LIKE 'PT' 
AND Subject  LIKE 'Poëzie'
AND title_ty LIKE 'h'
AND (Access LIKE 'ua' OR Access LIKE 'anet' )
group by URL

UNION

SELECT DISTINCT
token.cloi,
title_ti as Title,
subject_vw as Subject,
fulltext_access as Access,
fulltext_loc as Provider,
fulltext_zurl as URL
FROM token
JOIN 
fulltext ON token.cloi = fulltext.cloi
JOIN 
title ON fulltext.cloi = title.cloi
JOIN 
holding ON title.cloi = holding.cloi
JOIN 
subject ON subject.cloi = holding.cloi
JOIN 
impressum ON subject.cloi = impressum.cloi
WHERE
fulltext_ty LIKE 'full'
AND subject_ty LIKE 'PT' 
AND Subject  LIKE 'Verhalend proza'
AND title_ty LIKE 'h'
AND (Access LIKE 'ua' OR Access LIKE 'anet' )
group by URL

UNION

SELECT DISTINCT
token.cloi,
title_ti as Title,
subject_vw as Subject,
fulltext_access as Access,
fulltext_loc as Provider,
fulltext_zurl as URL
FROM token
JOIN 
fulltext ON token.cloi = fulltext.cloi
JOIN 
title ON fulltext.cloi = title.cloi
JOIN 
holding ON title.cloi = holding.cloi
JOIN 
subject ON subject.cloi = holding.cloi
JOIN 
impressum ON subject.cloi = impressum.cloi
WHERE
fulltext_ty LIKE 'full'
AND subject_ty LIKE 'PT' 
AND Subject  LIKE 'Beeldverhaal'
AND title_ty LIKE 'h'
AND (Access LIKE 'ua' OR Access LIKE 'anet' )
group by URL

UNION

SELECT DISTINCT
token.cloi,
title_ti as Title,
subject_vw as Subject,
fulltext_access as Access,
fulltext_loc as Provider,
fulltext_zurl as URL
FROM token
JOIN 
fulltext ON token.cloi = fulltext.cloi
JOIN 
title ON fulltext.cloi = title.cloi
JOIN 
holding ON title.cloi = holding.cloi
JOIN 
subject ON subject.cloi = holding.cloi
JOIN 
impressum ON subject.cloi = impressum.cloi
WHERE
fulltext_ty LIKE 'full'
AND subject_ty LIKE 'PT' 
AND Subject  LIKE 'Bloemlezing'
AND title_ty LIKE 'h'
AND (Access LIKE 'ua' OR Access LIKE 'anet' )
group by URL

'''         
description = ['Google books', 'Google books from Hendrik Conscience heritage library', 'Digital library for Dutch literature', 'Google books from Plantin-Moretus Museum', 'Delpher']
print_heading('The online text editions: the biggest providers (top 5)')
styling_frame_count(execute_cursor(query, ['cloi', 'title_ti','subject_vw', 'fulltext_access', 'fulltext_loc', 'fulltext_zurl']), 'fulltext_loc', 5)
print("\n \n \033[1m\033[31m" + f'Legenda \n \n google_book = \N{TAB} \N{TAB} Google books \n google_book_ehc = \N{TAB} Google books from Hendrik Conscience heritage library \n dbnl-tekst = \N{TAB} \N{TAB} Digital library for Dutch literature \n google_book_mpm = \N{TAB} Google books from Plantin-Moretus Museum \n delpher = \N{TAB} \N{TAB} Delpher books    \033[0m ')


 
 The online text editions: the biggest providers (top 5) 
  


,Number of records
google_book_ehc,438
google_book,375
dbnl-tekst,278
google_book_mpm,74
delpher,42



 
 Legenda 
 
 google_book = 	 	 Google books 
 google_book_ehc = 	 Google books from Hendrik Conscience heritage library 
 dbnl-tekst = 	 	 Digital library for Dutch literature 
 google_book_mpm = 	 Google books from Plantin-Moretus Museum 
 delpher = 	 	 Delpher books     


### Commentary on section 4

The digitization via Google projects score particularly well, but the Digital library for Dutch literature plays an important role for text editions in Dutch (works of Flemish and Dutch authors). 
This means that especially books that are copyright free are available. Recent content in the form of e-books is hardly present in the collection.



## 5 : German literature 

- Topic 9: Which German authors are the most common in the collection of text editions?
- Topic 10: The predominant language of these publications is obviously German, but in which other languages are these works published?  



In [6]:
# Query for Topic 9 and 10

query = '''
SELECT DISTINCT
token.cloi,
title_ti AS Title,
author_fn || ', ' || author_vn AS Author,
language_lg AS Language
FROM token
JOIN 
holding ON token.cloi = holding.cloi
JOIN 
title ON holding.cloi = title.cloi
JOIN 
subject ON subject.cloi = title.cloi
JOIN 
relation ON subject.cloi = relation.cloi
JOIN 
author on author.cloi = relation.cloi
JOIN 
subject2 ON author.cloi = subject2.cloi
JOIN 
language ON language.cloi = subject2.cloi
WHERE 
holding.lib LIKE 'UA-CST'
AND subject.subject_ac LIKE 'a::pt.41:1'
AND title_ty LIKE 'h'
AND subject2.subject_vw LIKE 'Duitse Letterkunde'
GROUP BY token.cloi

UNION

SELECT DISTINCT
token.cloi,
title_ti AS Title,
author_fn || ', ' || author_vn AS Author,
language_lg AS Language
FROM token
JOIN 
holding ON token.cloi = holding.cloi
JOIN 
title ON holding.cloi = title.cloi
JOIN 
subject ON subject.cloi = title.cloi
JOIN 
relation ON subject.cloi = relation.cloi
JOIN 
author on author.cloi = relation.cloi
JOIN 
subject2 ON author.cloi = subject2.cloi
JOIN 
language ON language.cloi = subject2.cloi
WHERE 
holding.lib LIKE 'UA-CST'
AND subject.subject_ac LIKE 'a::pt.42:1'
AND title_ty LIKE 'h'
AND subject2.subject_vw LIKE 'Duitse Letterkunde'
GROUP BY token.cloi

UNION

SELECT DISTINCT
token.cloi,
title_ti AS Title,
author_fn || ', ' || author_vn AS Author,
language_lg AS Language
FROM token
JOIN 
holding ON token.cloi = holding.cloi
JOIN 
title ON holding.cloi = title.cloi
JOIN 
subject ON subject.cloi = title.cloi
JOIN 
relation ON subject.cloi = relation.cloi
JOIN 
author on author.cloi = relation.cloi
JOIN 
subject2 ON author.cloi = subject2.cloi
JOIN 
language ON language.cloi = subject2.cloi
WHERE 
holding.lib LIKE 'UA-CST'
AND subject.subject_ac LIKE 'a::pt.43:1'
AND title_ty LIKE 'h'
AND subject2.subject_vw LIKE 'Duitse Letterkunde'
GROUP BY token.cloi

UNION

SELECT DISTINCT
token.cloi,
title_ti AS Title,
author_fn || ', ' || author_vn AS Author,
language_lg AS Language
FROM token
JOIN 
holding ON token.cloi = holding.cloi
JOIN 
title ON holding.cloi = title.cloi
JOIN 
subject ON subject.cloi = title.cloi
JOIN 
relation ON subject.cloi = relation.cloi
JOIN 
author on author.cloi = relation.cloi
JOIN 
subject2 ON author.cloi = subject2.cloi
JOIN 
language ON language.cloi = subject2.cloi
WHERE 
holding.lib LIKE 'UA-CST'
AND subject.subject_ac LIKE 'a::pt.46:1'
AND title_ty LIKE 'h'
AND subject2.subject_vw LIKE 'Duitse Letterkunde'
GROUP BY token.cloi

UNION

SELECT DISTINCT
token.cloi,
title_ti AS Title,
author_fn || ', ' || author_vn AS Author,
language_lg AS Language
FROM token
JOIN 
holding ON token.cloi = holding.cloi
JOIN 
title ON holding.cloi = title.cloi
JOIN 
subject ON subject.cloi = title.cloi
JOIN 
relation ON subject.cloi = relation.cloi
JOIN 
author on author.cloi = relation.cloi
JOIN 
subject2 ON author.cloi = subject2.cloi
JOIN 
language ON language.cloi = subject2.cloi
WHERE 
holding.lib LIKE 'UA-CST'
AND subject.subject_ac LIKE 'a::pt.47:1'
AND title_ty LIKE 'h'
AND subject2.subject_vw LIKE 'Duitse Letterkunde'
GROUP BY token.cloi

UNION

SELECT DISTINCT
token.cloi,
title_ti AS Title,
author_fn || ', ' || author_vn AS Author,
language_lg AS Language
FROM token
JOIN 
holding ON token.cloi = holding.cloi
JOIN 
title ON holding.cloi = title.cloi
JOIN 
subject ON subject.cloi = title.cloi
JOIN 
relation ON subject.cloi = relation.cloi
JOIN 
author on author.cloi = relation.cloi
JOIN 
subject2 ON author.cloi = subject2.cloi
JOIN 
language ON language.cloi = subject2.cloi
WHERE 
holding.lib LIKE 'UA-CST'
AND subject.subject_ac LIKE 'a::pt.17:1'
AND title_ty LIKE 'h'
AND subject2.subject_vw LIKE 'Duitse Letterkunde'
GROUP BY token.cloi

'''
print_heading('German authors of text editions (top 20)')
styling_frame_count(execute_cursor(query, ['cloi', 'title_ti','author_name', 'language_lg']), 'author_name', 20)
print_heading('Languages of German text editions')
styling_frame_count(execute_cursor(query, ['cloi', 'title_ti','author_name', 'language_lg']), 'language_lg', 5)


 
 German authors of text editions (top 20) 
  


,Number of records
"Grimm,",311
"Goethe, von, Johann Wolfgang",120
"Brecht, Bertolt",105
"Kafka, Franz",98
"Böll, Heinrich",83
"Schnitzler, Arthur",80
"Bernhard, Thomas",79
"Döblin, Alfred",71
"Mann, Thomas",71
"Handke, Peter",63



 
 Languages of German text editions 
  


,Number of records
ger,6432
dut,715
fre,526
eng,87
spa,17


### Commentary on section 5

#### Commentary on topic 9

If we consider the top 20, we can conclude that there is not a single 'contemporary' author, namely authors that have largely published the last ten or twenty years. This could point to the fact that in recent years no attention has been paid in the collection building to new novels or poems.

As already mentioned in section 1, the high ranking of the Brothers Grimm is due to the fable books collected by Wim Gielen, and especially due to the collection of children's literature donated by Johannes Bronkhorst (https://anet.be/record/opaccoloiuantwerpen/co:ua:155). 

#### Commentary on topic 10

There is a relatively large number of translations of German text editions into Dutch and French, as might be expected. On the contrary, English translations are less available.


## 6 : Year of publication 

- Topic 11: The aim of this section is to look at the year of publication. This is important to determine how the collection is grown over the years. The year of publication says nothing about the relevance of the text editions, but it could be in any case an indicator of the purchase policy of the faculty. 


In [7]:
# Query for Topic 11

query = '''
SELECT DISTINCT
    token.cloi,
    impressum_ju1sv as Year
FROM token 
JOIN 
author ON token.cloi = author.cloi
JOIN 
holding ON author.cloi = holding.cloi
JOIN 
title ON holding.cloi = title.cloi
JOIN 
subject ON subject.cloi = title.cloi
JOIN 
language ON subject.cloi = language.cloi
JOIN 
impressum ON impressum.cloi = language.cloi
WHERE 
subject_ty LIKE 'PT'
AND (subject_vw LIKE 'Theatertekst' OR subject_vw LIKE 'Tekstuitgave' OR subject_vw LIKE 'Poëzie' OR subject_vw LIKE 'Verhalend proza'
OR subject_vw LIKE 'Beeldverhaal' OR subject_vw LIKE 'Bloemlezing')
AND holding.lib LIKE 'UA-CST'
AND title_ty LIKE 'h'
GROUP BY token.cloi
'''
print_heading('Text editions: year of publication (top 30)')
styling_frame_count(execute_cursor(query, ['cloi', 'year']), 'year', 30)


 
 Text editions: year of publication (top 30) 
  


,Number of records
,3239
1980,1790
1977,1728
1976,1727
1975,1711
1974,1698
1995,1694
1994,1694
1972,1675
1978,1670


### Commentary on section 6

There is no year of publication of the period 2001-2021. This is once again a clear indication that little has been invested the last years in text editions (German literature). 